# Inventory Management
This notebook helps you manage your physical component inventory for KiCad projects.

In [1]:
# Load inventory management utilities
from inventory_manager import Inventory
from data_models.inventory_item import InventoryItem
import yaml
from pathlib import Path

In [2]:
# Load the inventory file
inv_path = Path('inventory.yaml')
inventory = Inventory(inv_path)
inventory.load()
print(f'Loaded {len(inventory.data.inventory_parts)} inventory items.')

Loaded 0 items from 'C:\Temp\KicadAutoFlow\inventory.yaml'.
Loaded 0 inventory items.


In [3]:
# Display all inventory items
import pandas as pd
df = inventory.to_dataframe()
df

,part_id,description,value,package,footprint,footprint_source,mpn,quantity,storage_location,datasheet_local,image_path


## Add a New Inventory Item
Edit and run the cell below to add a new part.

In [ ]:
# Example: Add a new inventory item
new_item = InventoryItem(
    part_id=inventory.get_next_part_id(),
    description='10k 0603 Resistor',
    value='10k',
    package='0603',
    footprint='Resistor_SMD:R_0603_1608Metric',
    footprint_source='manual',
    mpn=None,
    quantity=100
)
inventory.add_part(new_item)
inventory.save()
print('Added new item and saved inventory.')

## Add Inventory Item from a Picture
Upload a photo of your part, fill in the details, and add it to your inventory.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image
import os
img_upload = widgets.FileUpload(accept='image/*', multiple=False)
display(img_upload)

In [ ]:
# Save uploaded image and display it
if img_upload.value:
    img_info = next(iter(img_upload.value.values()))
    img_bytes = img_info['content']
    img_name = img_info['metadata']['name']
    img_dir = Path('docs/inventory_images')
    img_dir.mkdir(parents=True, exist_ok=True)
    img_path = img_dir / img_name
    with open(img_path, 'wb') as f:
        f.write(img_bytes)
    display(Image(filename=str(img_path)))
    print(f'Image saved to {img_path}')
else:
    print('Upload an image to continue.')

In [ ]:
# Fill in the fields for the new item
# (You can pre-fill these with AI/LLM suggestions if desired)
from ipywidgets import Text, IntText, Dropdown, Button, VBox
desc = Text(description='Description')
value = Text(description='Value')
package = Text(description='Package')
footprint = Text(description='Footprint')
quantity = IntText(description='Quantity', value=1)
add_btn = Button(description='Add to Inventory', button_style='success')
form = VBox([desc, value, package, footprint, quantity, add_btn])
display(form)

In [ ]:
# Add the new item to inventory when button is clicked
def on_add_clicked(b):
    if not img_upload.value:
        print('Please upload an image first.')
        return
    img_info = next(iter(img_upload.value.values()))
    img_name = img_info['metadata']['name']
    img_rel_path = f'docs/inventory_images/{img_name}'
    new_item = InventoryItem(
        part_id=inventory.get_next_part_id(),
        description=desc.value,
        value=value.value,
        package=package.value,
        footprint=footprint.value,
        footprint_source='manual',
        mpn=None,
        quantity=quantity.value,
        image_path=img_rel_path
    )
    inventory.add_part(new_item)
    inventory.save()
    print(f'Added {new_item.part_id} to inventory.')
add_btn.on_click(on_add_clicked)

## Add Inventory Item from a Picture with LMM Assistance
Upload a photo of your part, let the LMM suggest details, review, and add it to your inventory.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image
from pathlib import Path
img_upload = widgets.FileUpload(accept='image/*', multiple=False)
display(img_upload)

In [ ]:
# Save uploaded image and display it
img_path = None
if img_upload.value:
    img_info = next(iter(img_upload.value.values()))
    img_bytes = img_info['content']
    img_name = img_info['metadata']['name']
    img_dir = Path('docs/inventory_images')
    img_dir.mkdir(parents=True, exist_ok=True)
    img_path = img_dir / img_name
    with open(img_path, 'wb') as f:
        f.write(img_bytes)
    display(Image(filename=str(img_path)))
    print(f'Image saved to {img_path}')
else:
    print('Upload an image to continue.')

In [ ]:
# Use LMM/LLM to analyze the image and suggest part details
from adk_capabilities.llm_capability import LLMCapability
lmm_result = {}
if img_path and img_path.exists():
    llm = LLMCapability()  # Adjust if config/API key needed
    lmm_result = llm.analyze_image_for_component(str(img_path))
    print('LMM suggestion:', lmm_result)
else:
    print('Upload an image and rerun this cell.')

In [ ]:
# Pre-fill the form with LMM suggestions (user can edit)
from ipywidgets import Text, IntText, Button, VBox
desc = Text(description='Description', value=lmm_result.get('description', ''))
value = Text(description='Value', value=lmm_result.get('value', ''))
package = Text(description='Package', value=lmm_result.get('package', ''))
footprint = Text(description='Footprint', value=lmm_result.get('footprint', ''))
quantity = IntText(description='Quantity', value=1)
add_btn = Button(description='Add to Inventory', button_style='success')
form = VBox([desc, value, package, footprint, quantity, add_btn])
display(form)

In [ ]:
# Add the new item to inventory when button is clicked
def on_add_clicked(b):
    if not img_path or not img_path.exists():
        print('Please upload an image first.')
        return
    img_rel_path = str(img_path).replace('\\', '/').replace('\\', '/')
    new_item = InventoryItem(
        part_id=inventory.get_next_part_id(),
        description=desc.value,
        value=value.value,
        package=package.value,
        footprint=footprint.value,
        footprint_source='lmm' if lmm_result else 'manual',
        mpn=lmm_result.get('mpn', None),
        quantity=quantity.value,
        image_path=img_rel_path
    )
    inventory.add_part(new_item)
    inventory.save()
    print(f'Added {new_item.part_id} to inventory.')
add_btn.on_click(on_add_clicked)